In [1]:
import sys
sys.path.append('../src')

In [2]:
from covid import daily_snapshot
from pathlib import Path

import population as pop
import pandas as pd
import requests

In [3]:
pd.set_option('max_rows', 100) 

In [4]:
date = input('date in mm-dd-yyyy format')

date in mm-dd-yyyy format 04-03-2020


In [5]:
state = input('state, e.g. New Jersey')

state, e.g. New Jersey New Hampshire


### Quick glance at the data

In [6]:
df = daily_snapshot(date)
df.head()

,Unnamed: 0,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
FIPS,,,,,,,,,,,,
45001.0,NaN,Abbeville,South Carolina,US,2020-04-03 22:46:37,34.223334,-82.461707,6,0,0,0,"Abbeville, South Carolina, US"
22001.0,NaN,Acadia,Louisiana,US,2020-04-03 22:46:37,30.295065,-92.414197,72,1,0,0,"Acadia, Louisiana, US"
51001.0,NaN,Accomack,Virginia,US,2020-04-03 22:46:37,37.767072,-75.632346,8,0,0,0,"Accomack, Virginia, US"
16001.0,NaN,Ada,Idaho,US,2020-04-03 22:46:37,43.452658,-116.241552,307,3,0,0,"Ada, Idaho, US"
19001.0,NaN,Adair,Iowa,US,2020-04-03 22:46:37,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [7]:
df[df.Country_Region=='US'].describe()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active
count,0.0,2322.000000,2322.000000,2368.000000,2368.000000,2368.000000,2368.0
mean,NaN,37.952568,-90.418414,116.379223,2.992821,4.099240,0.0
std,NaN,5.119809,14.284283,1292.875036,39.369604,199.477621,0.0
min,NaN,13.444300,-159.596679,0.000000,0.000000,0.000000,0.0
25%,NaN,34.245714,-95.931390,2.000000,0.000000,0.000000,0.0
50%,NaN,38.020890,-88.213113,7.000000,0.000000,0.000000,0.0
75%,NaN,41.496883,-82.338627,27.000000,1.000000,0.000000,0.0
max,NaN,65.508155,145.673900,57159.000000,1584.000000,9707.000000,0.0


### Aggregate by state

In [8]:
state_totals = df[df.Country_Region=='US'].groupby(by='Province_State').sum()

In [9]:
state_totals.head()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active
Province_State,,,,,,,
Alabama,0.0,2139.512410,-5636.444765,1495,38,0,0
Alaska,0.0,549.064225,-1289.558627,157,3,0,0
Arizona,0.0,505.138555,-1671.948482,1937,41,0,0
Arkansas,0.0,1987.911393,-5266.351952,704,12,0,0
California,0.0,1915.653957,-6152.407712,12004,265,0,0


In [10]:
state_totals['death_rate?'] = state_totals['Deaths'] / state_totals['Confirmed'] * 100.0

In [11]:
state_totals.describe()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active,death_rate?
count,58.0,58.000000,58.000000,58.000000,58.000000,58.000000,58.0,57.000000
mean,0.0,1519.411428,-3619.854453,4751.482759,122.189655,167.362069,0.0,2.401692
std,0.0,1245.038349,3112.331438,14017.685794,395.557056,1274.591544,0.0,1.760712
min,0.0,0.000000,-14651.797929,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.0,511.030915,-5627.524695,336.500000,6.250000,0.000000,0.0,1.573677
50%,0.0,1287.901244,-3170.306929,944.500000,23.500000,0.000000,0.0,2.135493
75%,0.0,2515.280771,-958.649806,3405.750000,95.500000,0.000000,0.0,2.849874
max,0.0,4798.990598,145.673900,102987.000000,2935.000000,9707.000000,0.0,12.500000


In [12]:
state_covid = df[df.Province_State == state].copy()
state_covid['death_rate'] = 100 * state_covid['Deaths'] / state_covid['Confirmed']
state_columns=['Admin2', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'death_rate' ]

In [13]:
# sort_by = input('sort by one of Admin2, Confirmed, Deaths, death_rate')
sort_by = "Confirmed"

In [14]:
state_covid[state_columns].sort_values(by=sort_by, ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active,death_rate
FIPS,,,,,,
33015.0,Rockingham,170,1,0,0,0.588235
33011.0,Hillsborough,163,2,0,0,1.226994
33009.0,Grafton,38,0,0,0,0.000000
33017.0,Strafford,35,0,0,0,0.000000
33013.0,Merrimack,33,0,0,0,0.000000
33003.0,Carroll,14,0,0,0,0.000000
33001.0,Belknap,13,0,0,0,0.000000
33005.0,Cheshire,8,0,0,0,0.000000
33019.0,Sullivan,5,0,0,0,0.000000


## We need to locate county data to add per 1000 capita rates.

In [15]:
data = Path('../data/counties')
state_pop_path = data / Path(state.replace(' ', '').lower() + '.csv')

In [16]:
state_pop_path

PosixPath('../data/counties/newhampshire.csv')

In [17]:
state_pop = pd.read_csv(state_pop_path)

In [18]:
state_with_population = pd.merge(state_covid[state_columns], state_pop,
                                 how='outer',
                                 left_on='Admin2', right_on='county')

state_with_population['fraction_confirmed'] = state_with_population['Confirmed'] / state_with_population['pop2019']   * 1000.0
state_with_population['deaths']             = state_with_population['Deaths']    / state_with_population['pop2019']   * 1000.0
state_with_population['death_rate?']        = state_with_population['Deaths']    / state_with_population['Confirmed'] * 1000.0

# sort_by = input("column to sort Confirmed, Deaths, county, pop2019, fraction_confirmed, deaths, death_rate?")
sort_by = 'fraction_confirmed'
state_with_population.sort_values(by=sort_by, ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active,death_rate,Rank,county,pop2019,fraction_confirmed,deaths,death_rate?
6,Rockingham,170.0,1.0,0.0,0.0,0.588235,2.0,Rockingham,305129.0,0.557141,0.003277,5.882353
3,Grafton,38.0,0.0,0.0,0.0,0.000000,5.0,Grafton,89811.0,0.423111,0.000000,0.000000
4,Hillsborough,163.0,2.0,0.0,0.0,1.226994,1.0,Hillsborough,411087.0,0.396510,0.004865,12.269939
1,Carroll,14.0,0.0,0.0,0.0,0.000000,8.0,Carroll,47840.0,0.292642,0.000000,0.000000
7,Strafford,35.0,0.0,0.0,0.0,0.000000,4.0,Strafford,128237.0,0.272932,0.000000,0.000000
5,Merrimack,33.0,0.0,0.0,0.0,0.000000,3.0,Merrimack,149452.0,0.220807,0.000000,0.000000
0,Belknap,13.0,0.0,0.0,0.0,0.000000,7.0,Belknap,60640.0,0.214380,0.000000,0.000000
8,Sullivan,5.0,0.0,0.0,0.0,0.000000,9.0,Sullivan,43125.0,0.115942,0.000000,0.000000
2,Cheshire,8.0,0.0,0.0,0.0,0.000000,6.0,Cheshire,76263.0,0.104900,0.000000,0.000000
9,Unassigned,0.0,2.0,0.0,0.0,inf,NaN,NaN,NaN,NaN,NaN,inf


In [19]:
state_with_population.sort_values(by='Deaths', ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active,death_rate,Rank,county,pop2019,fraction_confirmed,deaths,death_rate?
4,Hillsborough,163.0,2.0,0.0,0.0,1.226994,1.0,Hillsborough,411087.0,0.396510,0.004865,12.269939
9,Unassigned,0.0,2.0,0.0,0.0,inf,NaN,NaN,NaN,NaN,NaN,inf
6,Rockingham,170.0,1.0,0.0,0.0,0.588235,2.0,Rockingham,305129.0,0.557141,0.003277,5.882353
0,Belknap,13.0,0.0,0.0,0.0,0.000000,7.0,Belknap,60640.0,0.214380,0.000000,0.000000
1,Carroll,14.0,0.0,0.0,0.0,0.000000,8.0,Carroll,47840.0,0.292642,0.000000,0.000000
2,Cheshire,8.0,0.0,0.0,0.0,0.000000,6.0,Cheshire,76263.0,0.104900,0.000000,0.000000
3,Grafton,38.0,0.0,0.0,0.0,0.000000,5.0,Grafton,89811.0,0.423111,0.000000,0.000000
5,Merrimack,33.0,0.0,0.0,0.0,0.000000,3.0,Merrimack,149452.0,0.220807,0.000000,0.000000
7,Strafford,35.0,0.0,0.0,0.0,0.000000,4.0,Strafford,128237.0,0.272932,0.000000,0.000000
8,Sullivan,5.0,0.0,0.0,0.0,0.000000,9.0,Sullivan,43125.0,0.115942,0.000000,0.000000


In [20]:
state_with_population.sort_values(by='death_rate', ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active,death_rate,Rank,county,pop2019,fraction_confirmed,deaths,death_rate?
9,Unassigned,0.0,2.0,0.0,0.0,inf,NaN,NaN,NaN,NaN,NaN,inf
4,Hillsborough,163.0,2.0,0.0,0.0,1.226994,1.0,Hillsborough,411087.0,0.396510,0.004865,12.269939
6,Rockingham,170.0,1.0,0.0,0.0,0.588235,2.0,Rockingham,305129.0,0.557141,0.003277,5.882353
0,Belknap,13.0,0.0,0.0,0.0,0.000000,7.0,Belknap,60640.0,0.214380,0.000000,0.000000
1,Carroll,14.0,0.0,0.0,0.0,0.000000,8.0,Carroll,47840.0,0.292642,0.000000,0.000000
2,Cheshire,8.0,0.0,0.0,0.0,0.000000,6.0,Cheshire,76263.0,0.104900,0.000000,0.000000
3,Grafton,38.0,0.0,0.0,0.0,0.000000,5.0,Grafton,89811.0,0.423111,0.000000,0.000000
5,Merrimack,33.0,0.0,0.0,0.0,0.000000,3.0,Merrimack,149452.0,0.220807,0.000000,0.000000
7,Strafford,35.0,0.0,0.0,0.0,0.000000,4.0,Strafford,128237.0,0.272932,0.000000,0.000000
8,Sullivan,5.0,0.0,0.0,0.0,0.000000,9.0,Sullivan,43125.0,0.115942,0.000000,0.000000
